# W10 — Matplotlib + Seaborn: Wizualizacja zaawansowana

**Programowanie w Pythonie II** | Politechnika Opolska

**Temat:** Seaborn — piękne wykresy, siatki subplotów i profesjonalne dashboardy

**Dataset:** tips — 244 rachunki z restauracji (wbudowany w seaborn)

## 0. Setup

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np

# Globalny motyw — wszystkie wykresy w tym stylu
sns.set_theme(style='whitegrid', palette='muted')

# Wczytaj dataset tips
tips = sns.load_dataset('tips')

print(f"Dataset tips: {tips.shape[0]} wierszy, {tips.shape[1]} kolumn")
print("\nTypy danych:")
print(tips.dtypes)
print("\nPierwsze 5 wierszy:")
tips.head()

## 1. Seaborn — podstawowe wykresy

### 1.1 barplot — średnia + przedziały ufności 95%

In [ ]:
# barplot — automatycznie liczy średnią i 95% CI (wąsy na słupkach)
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: średni rachunek per dzień, podział wg płci
sns.barplot(
    data=tips,
    x='day',
    y='total_bill',
    hue='sex',
    ax=axes[0],
    palette='muted'
)
axes[0].set_title('Średni rachunek wg dnia i płci', fontsize=12)
axes[0].set_xlabel('Dzień tygodnia')
axes[0].set_ylabel('Średni rachunek (USD)')

# Prawy: średni napiwek per dzień, podział wg pory dnia
sns.barplot(
    data=tips,
    x='day',
    y='tip',
    hue='time',
    ax=axes[1],
    palette='Set2'
)
axes[1].set_title('Średni napiwek wg dnia i pory', fontsize=12)
axes[1].set_xlabel('Dzień tygodnia')
axes[1].set_ylabel('Średni napiwek (USD)')

fig.suptitle('barplot — Seaborn automatycznie liczy 95% CI (wąsy)', fontsize=13)
plt.tight_layout()
plt.show()
plt.close()

### 1.2 boxplot — rozkład z percentylami

In [ ]:
# boxplot — Q1, mediana, Q3, IQR, outliers
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: rachunki per dzień, podział lunch/dinner
sns.boxplot(
    data=tips,
    x='day',
    y='total_bill',
    hue='time',
    ax=axes[0],
    palette='pastel'
)
axes[0].set_title('Rozkład rachunków wg dnia (lunch vs kolacja)', fontsize=11)
axes[0].set_xlabel('Dzień tygodnia')
axes[0].set_ylabel('Rachunek (USD)')

# Prawy: napiwki per dzień
sns.boxplot(
    data=tips,
    x='day',
    y='tip',
    ax=axes[1],
    palette='Set3'
)
axes[1].set_title('Rozkład napiwków wg dnia', fontsize=11)
axes[1].set_xlabel('Dzień tygodnia')
axes[1].set_ylabel('Napiwek (USD)')

fig.suptitle('boxplot — mediana, IQR i wartości odstające', fontsize=13)
plt.tight_layout()
plt.show()
plt.close()

### 1.3 violinplot — rozkład jako gęstość

In [ ]:
# violinplot — KDE + boxplot w jednym
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: rachunki lunch vs kolacja, split wg płci
sns.violinplot(
    data=tips,
    x='time',
    y='total_bill',
    hue='sex',
    split=True,
    ax=axes[0],
    palette='muted',
    inner='box'
)
axes[0].set_title('Rachunki: Lunch vs Kolacja (podział wg płci)', fontsize=11)
axes[0].set_xlabel('Pora dnia')
axes[0].set_ylabel('Rachunek (USD)')

# Prawy: napiwki per dzień, widok rozkładu
sns.violinplot(
    data=tips,
    x='day',
    y='tip',
    ax=axes[1],
    palette='Set2',
    inner='box'
)
axes[1].set_title('Rozkład napiwków wg dnia tygodnia', fontsize=11)
axes[1].set_xlabel('Dzień tygodnia')
axes[1].set_ylabel('Napiwek (USD)')

fig.suptitle('violinplot — kształt skrzypiec = zagęszczenie danych', fontsize=13)
plt.tight_layout()
plt.show()
plt.close()

### 1.4 heatmap — macierz korelacji i pivot

In [ ]:
# heatmap — dwa zastosowania: korelacja i pivot
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Lewy: korelacja zmiennych numerycznych
corr = tips.select_dtypes('number').corr()
sns.heatmap(
    corr,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    ax=axes[0],
    square=True
)
axes[0].set_title('Macierz korelacji zmiennych', fontsize=11)

# Prawy: pivot — średni rachunek per dzień i pora
pivot = tips.pivot_table(
    values='total_bill',
    index='day',
    columns='time',
    aggfunc='mean',
    observed=True
)
sns.heatmap(
    pivot,
    annot=True,
    fmt='.1f',
    cmap='YlOrRd',
    ax=axes[1]
)
axes[1].set_title('Średni rachunek (dzień × pora dnia)', fontsize=11)

fig.suptitle('heatmap — korelacja i pivot table jako kolory', fontsize=13)
plt.tight_layout()
plt.show()
plt.close()

### 1.5 pairplot — macierz wszystkich zmiennych

In [ ]:
# pairplot — scatter wszystkich par + rozkłady na przekątnej
# UWAGA: pairplot tworzy własną figurę — nie przyjmuje ax=
g = sns.pairplot(
    tips[['total_bill', 'tip', 'size', 'sex']],
    hue='sex',
    diag_kind='kde',
    plot_kws={'alpha': 0.6}
)
g.fig.suptitle('pairplot — wszystkie zmienne numeryczne vs płeć', y=1.02, fontsize=13)
plt.show()
plt.close()

## 2. Zaawansowany Matplotlib — siatki subplotów

### 2.1 plt.subplots — regularna siatka

In [ ]:
# Regularna siatka 2x3 — 6 wykresów
fig, axes = plt.subplots(
    2, 3,
    figsize=(15, 8),
    constrained_layout=True
)

# Rząd 0
sns.barplot(data=tips, x='day', y='total_bill', ax=axes[0, 0], palette='muted')
axes[0, 0].set_title('Barplot — średni rachunek')
axes[0, 0].set_xlabel('Dzień')
axes[0, 0].set_ylabel('Rachunek (USD)')

sns.boxplot(data=tips, x='day', y='tip', ax=axes[0, 1], palette='pastel')
axes[0, 1].set_title('Boxplot — napiwki')
axes[0, 1].set_xlabel('Dzień')
axes[0, 1].set_ylabel('Napiwek (USD)')

sns.violinplot(data=tips, x='time', y='total_bill', ax=axes[0, 2], palette='Set2')
axes[0, 2].set_title('Violinplot — rachunki')
axes[0, 2].set_xlabel('Pora dnia')
axes[0, 2].set_ylabel('Rachunek (USD)')

# Rząd 1
corr = tips.select_dtypes('number').corr()
sns.heatmap(corr, annot=True, fmt='.2f', ax=axes[1, 0], cbar=False, cmap='coolwarm')
axes[1, 0].set_title('Heatmap — korelacja')

sns.scatterplot(data=tips, x='total_bill', y='tip', hue='day', ax=axes[1, 1], alpha=0.7)
axes[1, 1].set_title('Scatter — rachunek vs napiwek')
axes[1, 1].set_xlabel('Rachunek (USD)')
axes[1, 1].set_ylabel('Napiwek (USD)')

sns.countplot(data=tips, x='day', hue='smoker', ax=axes[1, 2], palette='Set3')
axes[1, 2].set_title('Countplot — liczba wizyt')
axes[1, 2].set_xlabel('Dzień')
axes[1, 2].set_ylabel('Liczba wizyt')

fig.suptitle('Siatka 2×3 — plt.subplots(2, 3)', fontsize=15, fontweight='bold')
plt.show()
plt.close()

### 2.2 GridSpec — nieregularna siatka

In [ ]:
# GridSpec — elastyczna siatka z panelami różnej wielkości
fig = plt.figure(figsize=(15, 9), constrained_layout=True)
gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)

# Górny panel — rozciągnięty na 2 kolumny
ax_top = fig.add_subplot(gs[0, :2])
sns.barplot(
    data=tips, x='day', y='total_bill',
    hue='sex', ax=ax_top, palette='muted'
)
ax_top.set_title('Średni rachunek wg dnia i płci (panel główny — 2/3 szerokości)', fontsize=11)
ax_top.set_xlabel('Dzień tygodnia')
ax_top.set_ylabel('Rachunek (USD)')

# Górny prawy — mały pie chart
ax_tr = fig.add_subplot(gs[0, 2])
day_counts = tips['day'].value_counts()
ax_tr.pie(
    day_counts.values,
    labels=day_counts.index,
    autopct='%1.0f%%',
    colors=sns.color_palette('muted')[:len(day_counts)],
    startangle=90
)
ax_tr.set_title('Udział dni', fontsize=11)

# Środkowy rząd — trzy równe
ax_m1 = fig.add_subplot(gs[1, 0])
ax_m2 = fig.add_subplot(gs[1, 1])
ax_m3 = fig.add_subplot(gs[1, 2])

sns.boxplot(data=tips, x='time', y='tip', ax=ax_m1, palette='pastel')
ax_m1.set_title('Napiwki lunch/kolacja', fontsize=10)
ax_m1.set_xlabel('Pora dnia')
ax_m1.set_ylabel('Napiwek (USD)')

corr = tips.select_dtypes('number').corr()
sns.heatmap(corr, annot=True, fmt='.2f', ax=ax_m2, cbar=False, cmap='coolwarm')
ax_m2.set_title('Korelacja zmiennych', fontsize=10)

sns.scatterplot(
    data=tips, x='total_bill', y='tip',
    hue='smoker', ax=ax_m3, alpha=0.7
)
ax_m3.set_title('Rachunek vs napiwek', fontsize=10)
ax_m3.set_xlabel('Rachunek (USD)')
ax_m3.set_ylabel('Napiwek (USD)')

# Dolny — pełna szerokość
ax_bot = fig.add_subplot(gs[2, :])
tips_sum = tips.groupby('day', observed=True)['total_bill'].sum().reset_index()
ax_bot.bar(
    tips_sum['day'].astype(str),
    tips_sum['total_bill'],
    color=sns.color_palette('muted')[:4]
)
ax_bot.set_title('Suma rachunków wg dnia tygodnia (dolny panel — pełna szerokość)', fontsize=11)
ax_bot.set_xlabel('Dzień')
ax_bot.set_ylabel('Suma rachunków (USD)')

fig.suptitle('GridSpec 3×3 — panele różnej wielkości', fontsize=14, fontweight='bold')
plt.show()
plt.close()

### 2.3 Shared axes — wspólne osie

In [ ]:
# sharex='col' — kolumny dzielą oś X; sharey='row' — wiersze dzielą Y
fig, axes = plt.subplots(
    2, 2,
    figsize=(12, 8),
    sharex='col',
    sharey='row',
    constrained_layout=True
)

# Kolumna 0: napiwki
sns.stripplot(data=tips, x='day', y='tip', jitter=True, alpha=0.5, ax=axes[0, 0])
axes[0, 0].set_title('Strip plot — napiwki')
axes[0, 0].set_xlabel('')
axes[0, 0].set_ylabel('Napiwek (USD)')

sns.boxplot(data=tips, x='day', y='tip', ax=axes[1, 0], palette='pastel')
axes[1, 0].set_title('Box plot — napiwki')
axes[1, 0].set_xlabel('Dzień tygodnia')
axes[1, 0].set_ylabel('Napiwek (USD)')

# Kolumna 1: rachunki
sns.stripplot(data=tips, x='day', y='total_bill', jitter=True, alpha=0.5, ax=axes[0, 1])
axes[0, 1].set_title('Strip plot — rachunki')
axes[0, 1].set_xlabel('')
axes[0, 1].set_ylabel('Rachunek (USD)')

sns.boxplot(data=tips, x='day', y='total_bill', ax=axes[1, 1], palette='muted')
axes[1, 1].set_title('Box plot — rachunki')
axes[1, 1].set_xlabel('Dzień tygodnia')
axes[1, 1].set_ylabel('Rachunek (USD)')

fig.suptitle('Shared axes — ta sama skala Y per rząd, ta sama X per kolumnę', fontsize=13)
plt.show()
plt.close()

## 3. Dashboard — kompletna analiza na jednej figurze

### 3.1 Restaurant Analytics Dashboard — 6 paneli

In [ ]:
# Kompletny dashboard restauracji — 6 paneli
fig = plt.figure(figsize=(16, 11))
fig.patch.set_facecolor('#f8f9fa')

gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.45, wspace=0.35)

# === PANEL 1: Przychody per dzień — główny ===
ax1 = fig.add_subplot(gs[0, :2])
tips_day = tips.groupby('day', observed=True).agg(
    total=('total_bill', 'sum'),
    count=('total_bill', 'count')
).reset_index()
bars = ax1.bar(
    tips_day['day'].astype(str),
    tips_day['total'],
    color=sns.color_palette('muted')[:4],
    edgecolor='white',
    linewidth=1.2
)
for bar, val in zip(bars, tips_day['total']):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 20,
        f'${val:.0f}',
        ha='center', va='bottom', fontsize=9, fontweight='bold'
    )
ax1.set_title('Łączne przychody wg dnia tygodnia', fontsize=12, fontweight='bold')
ax1.set_xlabel('Dzień')
ax1.set_ylabel('Suma rachunków (USD)')
ax1.set_facecolor('#ffffff')

# === PANEL 2: Udział dni — kołowy ===
ax2 = fig.add_subplot(gs[0, 2])
day_counts = tips['day'].value_counts()
ax2.pie(
    day_counts.values,
    labels=day_counts.index,
    autopct='%1.0f%%',
    colors=sns.color_palette('muted')[:len(day_counts)],
    startangle=90
)
ax2.set_title('Udział wizyt wg dnia', fontsize=11, fontweight='bold')

# === PANEL 3: Boxplot napiwków ===
ax3 = fig.add_subplot(gs[1, 0])
sns.boxplot(
    data=tips, x='day', y='tip',
    palette='pastel', ax=ax3, linewidth=1.2
)
ax3.set_title('Rozkład napiwków wg dnia', fontsize=11, fontweight='bold')
ax3.set_xlabel('Dzień')
ax3.set_ylabel('Napiwek (USD)')
ax3.set_facecolor('#ffffff')

# === PANEL 4: Heatmapa korelacji ===
ax4 = fig.add_subplot(gs[1, 1])
corr = tips.select_dtypes('number').corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    ax=ax4,
    cbar=False,
    square=True
)
ax4.set_title('Korelacja zmiennych', fontsize=11, fontweight='bold')

# === PANEL 5: Scatter rachunek vs napiwek ===
ax5 = fig.add_subplot(gs[1, 2])
sns.scatterplot(
    data=tips,
    x='total_bill',
    y='tip',
    hue='smoker',
    style='time',
    alpha=0.7,
    ax=ax5,
    palette={'Yes': '#e74c3c', 'No': '#2ecc71'}
)
ax5.set_title('Rachunek vs napiwek', fontsize=11, fontweight='bold')
ax5.set_xlabel('Rachunek (USD)')
ax5.set_ylabel('Napiwek (USD)')
ax5.set_facecolor('#ffffff')
ax5.legend(fontsize=8, title_fontsize=8)

# === PANEL 6: Violinplot — dolny, pełna szerokość ===
ax6 = fig.add_subplot(gs[2, :])
sns.violinplot(
    data=tips,
    x='day',
    y='total_bill',
    hue='time',
    split=True,
    palette={'Lunch': '#3498db', 'Dinner': '#e74c3c'},
    ax=ax6,
    inner='box'
)
ax6.set_title(
    'Rozkład rachunków wg dnia i pory dnia (Lunch vs Dinner)',
    fontsize=12, fontweight='bold'
)
ax6.set_xlabel('Dzień tygodnia')
ax6.set_ylabel('Rachunek (USD)')
ax6.set_facecolor('#ffffff')

# Tytuł główny
fig.suptitle(
    'Restaurant Analytics Dashboard — Dataset Tips (244 rachunki)',
    fontsize=15, fontweight='bold', y=1.01
)

plt.show()
plt.close()

## 4. Stylizacja i eksport

### 4.1 Porównanie stylów Seaborn

In [ ]:
# Porównanie 4 stylów — context manager sns.axes_style()
styles = ['whitegrid', 'darkgrid', 'white', 'ticks']
fig, axes = plt.subplots(1, 4, figsize=(16, 3), constrained_layout=True)

for ax, style in zip(axes, styles):
    with sns.axes_style(style):
        sns.barplot(data=tips, x='day', y='total_bill', ax=ax, palette='muted')
        ax.set_title(f"style='{style}'", fontsize=10)
        ax.set_xlabel('')
        ax.set_ylabel('Rachunek' if ax == axes[0] else '')

fig.suptitle('Porównanie stylów Seaborn', fontsize=13)
plt.show()
plt.close()

### 4.2 Palety kolorów

In [ ]:
# Porównanie palet — 6 opcji
palettes = ['muted', 'bright', 'pastel', 'deep', 'Set2', 'colorblind']
fig, axes = plt.subplots(2, 3, figsize=(14, 6), constrained_layout=True)

for ax, palette in zip(axes.flat, palettes):
    sns.barplot(data=tips, x='day', y='total_bill', ax=ax, palette=palette)
    ax.set_title(f"palette='{palette}'", fontsize=10)
    ax.set_xlabel('')
    ax.set_ylabel('')

fig.suptitle('Palety kolorów Seaborn — porównanie', fontsize=13)
plt.show()
plt.close()

### 4.3 Eksport do pliku — savefig

In [ ]:
# Eksport do PNG i PDF
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(
    data=tips, x='day', y='total_bill',
    hue='sex', palette='muted', ax=ax
)
ax.set_title('Średni rachunek wg dnia i płci', fontsize=13)
ax.set_xlabel('Dzień tygodnia')
ax.set_ylabel('Rachunek (USD)')
sns.despine()  # usuwa górną i prawą krawędź osi

plt.tight_layout()

# Eksport PNG
plt.savefig(
    '/tmp/restauracja_analiza_demo.png',
    dpi=150,
    bbox_inches='tight',
    facecolor='white',
    format='png'
)
print("PNG zapisany: /tmp/restauracja_analiza_demo.png")

# Eksport PDF (wektorowy)
plt.savefig(
    '/tmp/restauracja_analiza_demo.pdf',
    dpi=300,
    bbox_inches='tight',
    format='pdf'
)
print("PDF zapisany: /tmp/restauracja_analiza_demo.pdf")

plt.show()
plt.close()

### 4.4 Adnotacja i legenda poza wykresem

In [ ]:
# Adnotacja strzałką na wykresie
tips_sum = tips.groupby('day', observed=True)['total_bill'].sum().reset_index()

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(
    tips_sum['day'].astype(str),
    tips_sum['total_bill'],
    color=sns.color_palette('muted')[:4]
)

# Znajdź szczyt
max_idx = tips_sum['total_bill'].idxmax()
max_day = str(tips_sum.loc[max_idx, 'day'])
max_val = tips_sum.loc[max_idx, 'total_bill']

# Adnotacja ze strzałką
ax.annotate(
    f'Szczyt: ${max_val:.0f}',
    xy=(max_idx, max_val),
    xytext=(max_idx + 0.6, max_val + 150),
    arrowprops=dict(arrowstyle='->', color='red', lw=1.8),
    fontsize=11, color='red', fontweight='bold'
)

ax.set_title('Łączne przychody wg dnia — z adnotacją szczytu', fontsize=13)
ax.set_xlabel('Dzień tygodnia')
ax.set_ylabel('Suma rachunków (USD)')
sns.despine()

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
# Legenda poza wykresem
fig, ax = plt.subplots(figsize=(9, 5))
sns.scatterplot(
    data=tips, x='total_bill', y='tip',
    hue='day', style='time', ax=ax, alpha=0.8
)

# Legenda poza wykresem — bbox_to_anchor
ax.legend(
    title='Dzień / Pora',
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    borderaxespad=0.
)

ax.set_title('Scatter: rachunek vs napiwek (legenda poza wykresem)', fontsize=12)
ax.set_xlabel('Rachunek (USD)')
ax.set_ylabel('Napiwek (USD)')

plt.tight_layout()
plt.show()
plt.close()

## 5. Wzorzec — funkcja do budowania wykresów

In [ ]:
# DRY — enkapsulacja wykresu w funkcji
def zbuduj_barplot(data, x, y, title, xlabel, ylabel, hue=None, palette='muted', filepath=None):
    """Buduje barplot Seaborn z pełnymi etykietami i opcjonalnym eksportem."""
    fig, ax = plt.subplots(figsize=(9, 5))
    sns.barplot(data=data, x=x, y=y, hue=hue, ax=ax, palette=palette)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    sns.despine()
    plt.tight_layout()
    if filepath:
        plt.savefig(filepath, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"Zapisano: {filepath}")
    plt.show()
    plt.close()

# Użycie — ten sam styl, różne dane
zbuduj_barplot(
    tips, 'day', 'total_bill',
    'Średni rachunek wg dnia tygodnia',
    'Dzień', 'Rachunek (USD)',
    hue='sex'
)

zbuduj_barplot(
    tips, 'day', 'tip',
    'Średni napiwek wg dnia tygodnia',
    'Dzień', 'Napiwek (USD)',
    hue='smoker',
    palette='Set2'
)

## 6. Podsumowanie — kluczowe komendy

```python
# === SEABORN — typy wykresów ===
sns.barplot(data=df, x='kat', y='num', hue='grupa', ax=ax)        # słupki + CI 95%
sns.boxplot(data=df, x='kat', y='num', hue='grupa', ax=ax)        # skrzynka z wąsami
sns.violinplot(data=df, x='kat', y='num', split=True, ax=ax)      # rozkład KDE
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=ax)  # mapa cieplna
sns.scatterplot(data=df, x='n1', y='n2', hue='kat', ax=ax)        # rozrzut
sns.countplot(data=df, x='kat', hue='kat2', ax=ax)                # zliczenie
g = sns.pairplot(df[cols], hue='kat')                              # macierz par

# === SUBPLOTS — regularna siatka ===
fig, axes = plt.subplots(2, 3, figsize=(14, 8), constrained_layout=True)
axes[0, 0]  # dostęp: [wiersz, kolumna]

# === GRIDSPEC — nieregularna siatka ===
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(3, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])   # cały pierwszy wiersz
ax = fig.add_subplot(gs[1, 0])   # konkretna komórka

# === STYL I EKSPORT ===
sns.set_theme(style='whitegrid', palette='muted')   # raz na początku
fig.suptitle('Tytuł', fontsize=14, fontweight='bold')
sns.despine()                                        # usuń górną i prawą krawędź
plt.tight_layout()                                   # lub constrained_layout=True
plt.savefig('plik.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()
plt.close()  # ZAWSZE na końcu każdej figury!
```